In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.fft import fft

### Loading Our Best SVR Model and Standard Scalar

In [2]:
filename = 'final_Best_Model.sav'
filename2 = 'Standard_scalar.sav'
Model = pickle.load(open(filename, 'rb'))
scalar = pickle.load(open(filename2, 'rb'))

## Predicting Test File

In [3]:
# list byteFiles directory
files = os.listdir('Test')

In [4]:
index = [i for i in range(len(files[2:]))]

In [5]:
Test = pd.DataFrame(index = index, columns = ['seg_id','10%_Roll_100','10%_Roll_1000', '5%_Roll_100', '5%_Roll_10000', '5%_Roll_1000', 'Mean', 'Real_FFT_Max', '10%_Roll_10000'])

In [6]:
def final_fun_1(files):
    for i in tqdm(range(len(files))):
        if i > 1:
            X = pd.read_csv('Test/' + files[i])
            X = X['acoustic_data']
            
            #Fast Fourier Transform
            ff_trans = fft(X)            #https://docs.scipy.org/doc/scipy/tutorial/fft.html
            #In Fast Fourier Transform we have real and imaginory data so with the help of numpy we distribute these data
            real_FFT = np.real(ff_trans)    #https://numpy.org/doc/stable/reference/generated/numpy.real.html
            Test['seg_id'][i-2] = files[i].split('.')[0]
            Test['Mean'][i-2] = X.mean()
            Test['Real_FFT_Max'][i-2] = real_FFT.max()
            for j in [100,1000,10000]:    #this will select window size as mention in the research paper
                Roll = X.rolling(j).std().dropna().values          #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
                if j == 100:
                    Test['5%_Roll_'+str(j)][i-2] = np.quantile(Roll, 0.05)
                    Test['10%_Roll_'+str(j)][i-2] = np.quantile(Roll, 0.10)
                elif j == 1000:
                    Test['5%_Roll_'+str(j)][i-2] = np.quantile(Roll, 0.05)
                    Test['10%_Roll_'+str(j)][i-2] = np.quantile(Roll, 0.10)
                elif j == 10000:
                    Test['5%_Roll_'+str(j)][i-2] = np.quantile(Roll, 0.05)
                    Test['10%_Roll_'+str(j)][i-2] = np.quantile(Roll, 0.10)
    test = Test[['10%_Roll_100','10%_Roll_1000', '5%_Roll_100', '5%_Roll_10000', '5%_Roll_1000', 'Mean', 'Real_FFT_Max', '10%_Roll_10000']]
    
    #scaler = StandardScaler()
    scalar.transform(test)
    test_scaled = pd.DataFrame(scalar.fit_transform(test), columns=test.columns)
    
    #For prediction we use SVR with top features model because it have less MAE as compare to others
    y_pred = Model.predict(test_scaled)
    
    return y_pred

'''
In this function I first load my test data which have different csv file of 150000 rows each. Then I do feature engineering based on
top features I take during training the model and than I apply our SVR model for prediction and return the whole predicted array.
'''
    

'\nIn this function I first load my test data which have different csv file of 150000 rows each. Then I do feature engineering based on\ntop features I take during training the model and than I apply our SVR model for prediction and return the whole predicted array.\n'

In [7]:
y_pred = final_fun_1(files)

100%|██████████| 2626/2626 [02:32<00:00, 17.23it/s]


### We can't build function 2 as mention in the instruction because we don't have true Target value to compute MAE (Mean Absolute Error).

In [8]:
submission = Test[['seg_id']]

In [9]:
submission['time_to_failure'] = y_pred

C:\Users\manas\Anaconda3\envs\Tensor_GPU\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
submission

,seg_id,time_to_failure
0,seg_00030f,4.533960
1,seg_0012b5,4.899752
2,seg_00184e,5.756910
3,seg_003339,7.982026
4,seg_0042cc,6.723822
...,...,...
2619,seg_ff4236,5.188404
2620,seg_ff7478,6.360181
2621,seg_ff79d9,3.048874
2622,seg_ffbd6a,1.534785


In [11]:
submission.to_csv('submission.csv', index=False)   #https://stackoverflow.com/questions/20107570/removing-index-column-in-pandas-when-reading-a-csv

## Result
#### On Kaggle after submission I got a Public Score: 1.690 and a Private Score: 2.606